# DD-Pose
## Create example images and videos for website

In [ ]:
from dd_pose.dataset import Dataset
from dd_pose.dataset_item import DatasetItem
from dd_pose.image_decorator import ImageDecorator
from dd_pose.jupyter_helpers import showimage
from dd_pose.visualization_helpers import get_dashboard

from multiprocess import Pool
import numpy as np
import random
import cv2
import os

In [ ]:
d = Dataset(split='all')
len(d)

### Create carpet image with with one image per subject

In [ ]:
image_width = 256
image_height = 256

images = []
random.seed(a=0xbeef * 8)
for subject, scenario_dict in sorted(d.data.items()):
    scenario_names = scenario_dict.keys()
    random_scenario_name = scenario_names[random.randint(0, len(scenario_names) - 1)]

    di_dicts = scenario_dict[random_scenario_name]
    random_di_dict = di_dicts[random.randint(0, len(di_dicts) - 1)]
    
    di = DatasetItem(random_di_dict)
    stamps = di.get_stamps()
    stamp = stamps[random.randint(0, len(stamps) - 1)]
    img, _ = di.get_img_driver_left(stamp, shift=True)
    images.append(cv2.resize(img, (image_height, image_width), interpolation=cv2.INTER_AREA))

In [ ]:
# lay out images in carpet
n_cols = 7
n_rows = 4
assert(len(images) <= n_cols * n_rows)
carpet_image = np.zeros((n_rows*image_height, n_cols*image_width), dtype=np.uint8)
for i, image in enumerate(images):
    row_index = i // n_cols
    col_index = i % n_cols
    row_px = row_index * image_height
    col_px = col_index * image_width

    carpet_image[row_px:row_px+image_height, col_px:col_px+image_width] = image

In [ ]:
showimage(carpet_image)

In [ ]:
cv2.imwrite(os.path.join('resources', 'all-subjects-carpet.png'), carpet_image)

### Create timeslice image with head pose and dashboard

In [ ]:
di_dict = d.get(subject_id=7, scenario_id=7, humanhash='kentucky-sweet-mars-high')
di = DatasetItem(di_dict)
stamps = di.get_stamps()

In [ ]:
timeslices = []

image_width = 256
image_height = 256

start = 1345
n_slices = 10
step = 25
for i in range(n_slices):
    stamp = stamps[start + i * step]
    img, pcm = di.get_img_driver_left(stamp, shift=True)
    img_color = np.dstack((img, img, img))
    image_decorator = ImageDecorator(img_color, pcm)
    image_decorator.draw_axis(di.get_T_camdriver_head(stamp))
    img_driver = cv2.resize(img_color, (image_width, image_height), interpolation=cv2.INTER_AREA)
    img_driver = img_driver[25:-25,:]
    # showimage(img_driver)
    img, pcm = di.get_img_docu(stamp)
    image_decorator = ImageDecorator(img, pcm)
    image_decorator.draw_axis(np.linalg.inv(di.get_T_camdriver_camdocu()).dot(di.get_T_camdriver_head(stamp)))
    img_docu = cv2.resize(img, (image_width, int(float(image_width)*img.shape[0]/img.shape[1])), interpolation=cv2.INTER_AREA)
    # showimage(img_docu)
    timeslice = np.vstack((img_driver, img_docu))
    timeslices.append(timeslice)

In [ ]:
carpet_image = np.vstack((np.hstack(timeslices[0:len(timeslices)/2]), np.hstack(timeslices[len(timeslices)/2:])))
showimage(carpet_image)
carpet_image.shape

In [ ]:
cv2.imwrite(os.path.join('resources', 'driver-and-docu-timeslice.png'), carpet_image)

### Create carpet showing challenges

In [ ]:
image_width = 80
image_height = image_width

images = []
random.seed(a=0xbeef * 8)
di_dicts = list(d.get_dataset_items())
len(di_dicts)

In [ ]:
n_cols = 16
n_rows = 9
images = []
random_di_dicts = np.random.choice(di_dicts, n_rows * n_cols)
for random_di_dict in random_di_dicts:

    di = DatasetItem(random_di_dict)
    stamps = di.get_stamps()
    stamp = stamps[random.randint(0, len(stamps) - 1)]
    img, _ = di.get_img_driver_left(stamp, shift=True)
    images.append(cv2.resize(img, (image_height, image_width), interpolation=cv2.INTER_AREA))

In [ ]:
assert(len(images) <= n_cols * n_rows)
carpet_image = np.zeros((n_rows*image_height, n_cols*image_width), dtype=np.uint8)
for i, image in enumerate(images):
    row_index = i // n_cols
    col_index = i % n_cols
    row_px = row_index * image_height
    col_px = col_index * image_width

    carpet_image[row_px:row_px+image_height, col_px:col_px+image_width] = image

In [ ]:
showimage(carpet_image)

In [ ]:
cv2.imwrite(os.path.join('resources', 'carpet-challenges.png'), carpet_image)

### Get dashboard image with blurred gps

In [ ]:
random.seed(a=85)
di_dicts = list(d.get_dataset_items())
di = DatasetItem(random.choice(di_dicts))
stamp = random.choice(di.get_stamps())

dashboard_img = get_dashboard(di, stamp)

In [ ]:
def blur_dashboard_gps(dashboard_img):
    row = 650
    col = 1646
    width = 300
    height = 100
    dashboard_img[row:row+height, col:col+width] = cv2.GaussianBlur(dashboard_img[row:row+height, col:col+width], (11,11), 100)
    return dashboard_img

In [ ]:
dashboard_img = blur_dashboard_gps(dashboard_img)
showimage(dashboard_img)

In [ ]:
cv2.imwrite(os.path.join('resources', 'dashboard-gps-blurred.png'), dashboard_img)

### Create first example dashboard video

In [ ]:
def write_dashboard_image(png_output_dir, di, stamp):
    dashboard_img = get_dashboard(di, stamp)
    dashboard_img = blur_dashboard_gps(dashboard_img)
    cv2.imwrite(os.path.join(png_output_dir, '%ld.png' % stamp), dashboard_img)    

In [ ]:
di_dict = d.get(subject_id=7, scenario_id=7, humanhash='kentucky-sweet-mars-high')
di = DatasetItem(di_dict)
dataset_item_str = d.get_dataset_item_str(di_dict)
png_output_dir = os.path.join('/tmp', dataset_item_str)
try:
    os.makedirs(png_output_dir)
except:
    pass

pool = Pool(16)
pool.map(lambda stamp: write_dashboard_image(png_output_dir, di, stamp), di.get_stamps())
del pool

In [ ]:
print('mencoder -ovc lavc -oac lavc -of lavf -lavfopts format=webm -lavcopts threads=4:acodec=vorbis:vcodec=libvpx -ffourcc VP80 mf:///tmp/%s/*.png -mf fps=15:type=png -o %s.webm' % (dataset_item_str, dataset_item_str))

### Create second example dashboard video

In [ ]:
di_dict = d.get(subject_id=19, scenario_id=5, humanhash='juliet-stairway-kitten-saturn')
di = DatasetItem(di_dict)
dataset_item_str = d.get_dataset_item_str(di_dict)
png_output_dir = os.path.join('/tmp', dataset_item_str)
print(png_output_dir)
try:
    os.makedirs(png_output_dir)
except:
    pass

In [ ]:
pool = Pool(16)
pool.map(lambda stamp: write_dashboard_image(png_output_dir, di, stamp), di.get_stamps()[:15*60])
del pool

In [ ]:
print('mencoder -ovc lavc -oac lavc -of lavf -lavfopts format=webm -lavcopts threads=4:acodec=vorbis:vcodec=libvpx -ffourcc VP80 mf:///tmp/%s/*.png -mf fps=15:type=png -o %s.webm' % (dataset_item_str, dataset_item_str))